In [3]:
#!/usr/bin/env python 

'''
-Initially make z_to_x and x_to_z fairly shallow networks.  Inject noise?  

-Use the fflayer class?  

'''
import sys

sys.path.append("/u/lambalex/DeepLearning/undirected_matching")
sys.path.append("/u/lambalex/DeepLearning/undirected_matching/lib")

import theano
import theano.tensor as T
from nn_layers import fflayer, param_init_fflayer, param_init_convlayer, convlayer
from utils import init_tparams, join2, srng, dropout, inverse_sigmoid, join3, merge_images
from loss import accuracy, crossent, lsgan_loss, wgan_loss, improvement_loss
import lasagne
import numpy as np
import numpy.random as rng
import gzip
import cPickle as pickle
import random
from viz import plot_images
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

import os
slurm_name = os.environ["SLURM_JOB_ID"]

Using cuDNN version 5105 on context None
Mapped name None to device cuda0: Quadro K6000 (0000:04:00.0)
/Tmp/lisa/os_v5/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Tmp/lisa/os_v5/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:1401: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [4]:
class ConsiderConstant(theano.compile.ViewOp):
    def grad(self, args, g_outs):
        return [T.zeros_like(g_out) for g_out in g_outs]

consider_constant = ConsiderConstant()

#dataset = "mnist"
#dataset = "anime"
dataset = "svhn"

if dataset == "mnist":
    mn = gzip.open("/u/lambalex/data/mnist/mnist.pkl.gz")

    train, valid, test = pickle.load(mn)

    trainx,trainy = train
    
    
    #newtx = trainx[(trainy<2) | (trainy>8)]
    #newty = trainy[(trainy<2) | (trainy>8)]
    #trainx = newtx
    #trainy = newty
    
    validx,validy = valid
    testx, testy = test

    num_examples = trainx.shape[0]

    m = 784
elif dataset == "anime":
    from load_file import FileData, normalize, denormalize

    loc = "/u/lambalex/DeepLearning/animefaces/datafaces/danbooru-faces/"

    animeData = FileData(loc, 32, 64)

    m = 32*32*3

elif dataset == "svhn":

    from load_svhn import SvhnData
    from load_file import normalize, denormalize

    svhnData = SvhnData(mb_size=64,segment="train")

    num_examples = 50000

nl = 128
print "num latent", nl
#128 works for nl
nfg = 512
nfd = 512

print "dataset", dataset

#3
num_steps = 1
print "num steps", num_steps

train_classifier_separate = True
print "train classifier separate", train_classifier_separate

#skip_conn = True
#print "skip conn", skip_conn

latent_sparse = False
print "latent sparse", latent_sparse

persist_p_chain = False
print "persistent p chain", persist_p_chain

blending_rate = 0.5
print 'blending rate (odds of keeping old z in P chain)', blending_rate

improvement_loss_weight = 0.0
print "improvement loss weight", improvement_loss_weight

shape (574168, 3, 32, 32)
svhn shape (574168, 3, 32, 32)
0 255
num latent 128
dataset svhn
num steps 1
train classifier separate True
latent sparse False
persistent p chain False
blending rate (odds of keeping old z in P chain) 0.5
improvement loss weight 0.0


In [5]:
def init_gparams(p):

    p = param_init_fflayer(options={},params=p,prefix='z_x_1',nin=nl*2,nout=512*4*4,ortho=False,batch_norm=True)

    p = param_init_convlayer(options={},params=p,prefix='z_x_2',nin=512,nout=256,kernel_len=5,batch_norm=True)
    p = param_init_convlayer(options={},params=p,prefix='z_x_3',nin=256*1,nout=128,kernel_len=5,batch_norm=True)
    p = param_init_convlayer(options={},params=p,prefix='z_x_4',nin=128*1,nout=3,kernel_len=5,batch_norm=False)

    p = param_init_convlayer(options={},params=p,prefix='x_z_1',nin=3,nout=128,kernel_len=5,batch_norm=True)
    p = param_init_convlayer(options={},params=p,prefix='x_z_2',nin=128,nout=256,kernel_len=5,batch_norm=True)
    p = param_init_convlayer(options={},params=p,prefix='x_z_3',nin=256,nout=512,kernel_len=5,batch_norm=True)

    p = param_init_fflayer(options={},params=p,prefix='x_z_mu',nin=512*4*4,nout=nl,ortho=False,batch_norm=False)
    p = param_init_fflayer(options={},params=p,prefix='x_z_sigma',nin=512*4*4,nout=nl,ortho=False,batch_norm=False)

    return init_tparams(p)

def init_dparams(p):

    p = param_init_convlayer(options={},params=p,prefix='DC_1',nin=3,nout=128,kernel_len=5,batch_norm=False)
    p = param_init_convlayer(options={},params=p,prefix='DC_2',nin=128,nout=256,kernel_len=5,batch_norm=False)
    p = param_init_convlayer(options={},params=p,prefix='DC_3',nin=256,nout=512,kernel_len=5,batch_norm=False)

    p = param_init_fflayer(options={},params=p,prefix='D_1',nin=nl+512*4*4,nout=nfd,ortho=False,batch_norm=False)
    p = param_init_fflayer(options={},params=p,prefix='D_2',nin=nfd,nout=nfd,ortho=False,batch_norm=False)
    p = param_init_fflayer(options={},params=p,prefix='D_3',nin=nfd,nout=nfd,ortho=False,batch_norm=False)

    p = param_init_fflayer(options={},params=p,prefix='D_o_1',nin=nfd,nout=1,ortho=False,batch_norm=False)
    p = param_init_fflayer(options={},params=p,prefix='D_o_2',nin=nfd,nout=1,ortho=False,batch_norm=False)
    p = param_init_fflayer(options={},params=p,prefix='D_o_3',nin=nfd,nout=1,ortho=False,batch_norm=False)

    p = param_init_convlayer(options={},params=p,prefix='D_o_4',nin=128,nout=1,kernel_len=5,batch_norm=False)
    p = param_init_convlayer(options={},params=p,prefix='D_o_5',nin=256,nout=1,kernel_len=5,batch_norm=False)
    p = param_init_convlayer(options={},params=p,prefix='D_o_6',nin=512,nout=1,kernel_len=5,batch_norm=False)

    return init_tparams(p)


def z_to_x(p,z):

    print "extra noise input"
    z_inp = join2(z, 1.0*srng.normal(size=z.shape))

    d0 = fflayer(tparams=p,state_below=z_inp,options={},prefix='z_x_1',activ='lambda x: tensor.nnet.relu(x,alpha=0.02)')

    d0 = d0.reshape((64,512,4,4))

    d1 = convlayer(tparams=p,state_below=d0,options={},prefix='z_x_2',activ='lambda x: tensor.nnet.relu(x,alpha=0.02)',stride=-2)

    d2 = convlayer(tparams=p,state_below=d1,options={},prefix='z_x_3',activ='lambda x: tensor.nnet.relu(x,alpha=0.02)',stride=-2)

    d3 = convlayer(tparams=p,state_below=d2,options={},prefix='z_x_4',activ='lambda x: x',stride=-2)

    x_new = d3.flatten(2)

    return x_new

def x_to_z(p,x):

    e1 = convlayer(tparams=p,state_below=x.reshape((64,3,32,32)),options={},prefix='x_z_1',activ='lambda x: tensor.nnet.relu(x,alpha=0.02)',stride=2)

    e2 = convlayer(tparams=p,state_below=e1,options={},prefix='x_z_2',activ='lambda x: tensor.nnet.relu(x,alpha=0.02)',stride=2)

    e3 = convlayer(tparams=p,state_below=e2,options={},prefix='x_z_3',activ='lambda x: tensor.nnet.relu(x,alpha=0.02)',stride=2)

    eo = e3
    eo = eo.flatten(2)

    sigma = fflayer(tparams=p,state_below=eo,options={},prefix='x_z_mu',activ='lambda x: x')
    mu = fflayer(tparams=p,state_below=eo,options={},prefix='x_z_sigma',activ='lambda x: x')

    eps = srng.normal(size=sigma.shape)

    z_new = eps*T.nnet.sigmoid(sigma) + mu
    print "turned on injected noise in x->z connection"

    z_new = (z_new - T.mean(z_new, axis=0, keepdims=True)) / (0.001 + T.std(z_new, axis=0, keepdims=True))

    return z_new, eo


def discriminator(p,x,z):

    dc_1 = convlayer(tparams=p,state_below=x.reshape((64,3,32,32)),options={},prefix='DC_1',activ='lambda x: tensor.nnet.relu(x,alpha=0.02)',stride=2)

    dc_2 = convlayer(tparams=p,state_below=dc_1,options={},prefix='DC_2',activ='lambda x: tensor.nnet.relu(x,alpha=0.02)',stride=2)

    dc_3 = convlayer(tparams=p,state_below=dc_2,options={},prefix='DC_3',activ='lambda x: tensor.nnet.relu(x,alpha=0.02)',stride=2)

    inp = join2(z,dc_3.flatten(2))

    h1 = fflayer(tparams=p,state_below=inp,options={},prefix='D_1',activ='lambda x: tensor.nnet.relu(x,alpha=0.02)',mean_ln=False)

    h2 = fflayer(tparams=p,state_below=h1,options={},prefix='D_2',activ='lambda x: tensor.nnet.relu(x,alpha=0.02)', mean_ln=False)

    h3 = fflayer(tparams=p,state_below=h2,options={},prefix='D_3',activ='lambda x: tensor.nnet.relu(x,alpha=0.02)', mean_ln=False)

    D1 = fflayer(tparams=p,state_below=h1,options={},prefix='D_o_1',activ='lambda x: x')
    D2 = fflayer(tparams=p,state_below=h2,options={},prefix='D_o_2',activ='lambda x: x')
    D3 = fflayer(tparams=p,state_below=h3,options={},prefix='D_o_3',activ='lambda x: x')

    D4 = convlayer(tparams=p,state_below=dc_1,options={},prefix='D_o_4',activ='lambda x: x',stride=2)
    D5 = convlayer(tparams=p,state_below=dc_2,options={},prefix='D_o_5',activ='lambda x: x',stride=2)
    D6 = convlayer(tparams=p,state_below=dc_3,options={},prefix='D_o_6',activ='lambda x: x',stride=2)

    print "special thing in D"
    return [D1,D2,D3,D4,D5,D6], h3

def p_chain(p, z, num_iterations):
    zlst = [z]
    xlst = []

    if num_iterations == 1:
        
        new_x = z_to_x(p, zlst[-1])
        xlst.append(new_x)
        #new_z = x_to_z(p, xlst[-1])
        #zlst.append(new_z)

    elif num_iterations == 3:  

        new_x = z_to_x(p, zlst[-1])
        xlst.append(new_x)
        new_z = x_to_z(p, consider_constant(xlst[-1]))
        zlst.append(new_z)

        new_x = z_to_x(p, zlst[-1])
        xlst.append(new_x)
        new_z = x_to_z(p, consider_constant(xlst[-1]))
        zlst.append(new_z)

        new_x = z_to_x(p, zlst[-1])
        xlst.append(new_x)

    else:

        for inds in range(0,num_iterations):
            new_x = z_to_x(p, zlst[-1])
            xlst.append(new_x)
            new_z, eo = x_to_z(p, xlst[-1])
            zlst.append(new_z)


    for j in range(len(xlst)):
        xlst[j] = T.nnet.sigmoid(xlst[j])

    return xlst, zlst

def onestep_z_to_x(p,z):
    x = T.nnet.sigmoid(z_to_x(p, z))
    return x

def onestep_x_to_z(p,x):
    new_z, eo = x_to_z(p, inverse_sigmoid(x))
    return new_z, eo

def q_chain(p,x,num_iterations):

    xlst = [x]
    zlst = []
    new_z, eo = x_to_z(p, inverse_sigmoid(xlst[-1]))
    zlst.append(new_z)

    return xlst, zlst

In [6]:
import pickle, os

directory = "network-temp/"
ext = "svhn-soa.p"

if not os.path.exists(directory):
    os.makedirs(directory)
    
def save_network(gparams,dparams,name):
    pkl_params = (gparams,dparams)
    out = open(directory + str(name) + ext, "w", 0) #bufsize=0
    pickle.dump(pkl_params, out)
    out.close()

def load_network(name):
    return pickle.load(open(directory + str(name) + ext, "r" ))

In [7]:
# gparams = init_gparams({})
# dparams = init_dparams({})

gparams,dparams = load_network("test")

In [8]:
z_in = T.matrix('z_in')
x_in = T.matrix()

p_lst_x,p_lst_z = p_chain(gparams, z_in, num_steps)

q_lst_x,q_lst_z = q_chain(gparams, x_in, num_steps)

p_lst_x_long,p_lst_z_long = p_chain(gparams, z_in, 19)

z_inf = q_lst_z[-1]

print p_lst_x
print p_lst_z
print q_lst_x
print q_lst_z

#D_p_lst_3,_ = discriminator(dparams, p_lst_x[2], p_lst_z[2])

#D_p_lst_2,_ = discriminator(dparams, p_lst_x[1], p_lst_z[1])

D_p_lst_1,_ = discriminator(dparams, p_lst_x[0], p_lst_z[0])

D_q_lst,D_feat_q = discriminator(dparams, q_lst_x[-1], q_lst_z[-1])

dloss, gloss = lsgan_loss(D_q_lst, D_p_lst_1)

print "single disc"
print "not using improvement objective"
#improvement_objective = improvement_loss_weight * improvement_loss(D_p_lst_1, D_p_lst_2)
#gloss += improvement_objective

lr = theano.shared(np.array(0.0001, dtype=theano.config.floatX))

dupdates = lasagne.updates.rmsprop(dloss, dparams.values(),lr)
gloss_grads = T.grad(gloss, gparams.values(), disconnected_inputs='ignore')
gupdates = lasagne.updates.rmsprop(gloss_grads, gparams.values(),lr)

gcupdates = lasagne.updates.rmsprop(gloss, gparams.values(),lr)

dgupdates = dupdates.copy()
dgupdates.update(gupdates)

dgcupdates = dupdates.copy()
dgcupdates.update(gcupdates)

train_disc_gen_classifier = theano.function(inputs = [x_in, z_in], outputs=[dloss,p_lst_x[-1],p_lst_z[-1]], updates=dgcupdates,on_unused_input='ignore')

train_disc_gen_classifier_fast = theano.function(inputs = [x_in, z_in], outputs=[dloss], updates=dgcupdates,on_unused_input='ignore')


get_zinf = theano.function([x_in], outputs=z_inf)
#get_dfeat = theano.function([x_in], outputs=D_feat_q)

#get_pchain = theano.function([z_in], outputs = p_lst_x_long)

x_in = T.matrix()

func_z_to_x = theano.function([z_in], outputs = onestep_z_to_x(gparams, z_in))
func_x_to_z = theano.function([x_in], outputs = onestep_x_to_z(gparams, x_in))



extra noise input


WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Quadro K6000 (CNMeM is disabled, cuDNN 5105)


turned on injected noise in x->z connection
extra noise input
turned on injected noise in x->z connection
extra noise input
turned on injected noise in x->z connection
extra noise input
turned on injected noise in x->z connection
extra noise input
turned on injected noise in x->z connection
extra noise input
turned on injected noise in x->z connection
extra noise input
turned on injected noise in x->z connection
extra noise input
turned on injected noise in x->z connection
extra noise input
turned on injected noise in x->z connection
extra noise input
turned on injected noise in x->z connection
extra noise input
turned on injected noise in x->z connection
extra noise input
turned on injected noise in x->z connection
extra noise input
turned on injected noise in x->z connection
extra noise input
turned on injected noise in x->z connection
extra noise input
turned on injected noise in x->z connection
extra noise input
turned on injected noise in x->z connection
extra noise input
turned o

In [9]:
print "a"

a


In [10]:
lr.set_value(0.0001)

In [11]:
z_out_p = rng.normal(size=(64,nl)).astype('float32')

for iteration in range(0,500000):

    if persist_p_chain:
        z_in_new = rng.normal(size=(64,nl)).astype('float32')
        blending = rng.uniform(0.0,1.0,size=(64,))
        z_in_new[blending>=blending_rate] = z_out_p[blending>=blending_rate]
        z_in = z_in_new
    else:
        z_in = rng.normal(size=(64,nl)).astype('float32')

    if latent_sparse:
        z_in[:,128:] *= 0.0

    r = random.randint(0,num_examples-64)

    if dataset == "mnist":
        x_in = trainx[r:r+64]
        y_in = trainy[r:r+64]

        x_in = x_in.reshape((64,1,28,28))

        x_in = np.repeat(x_in,3,axis=(1))
        x_in = np.lib.pad(x_in,((0,0),(0,0),(2,2),(2,2)),'constant',constant_values=(0))

        x_in = x_in.reshape((64,32*32*3))
    elif dataset == "anime":
        x_in = normalize(animeData.getBatch()).reshape((64,32*32*3))

    elif dataset == "svhn":
        x_in = normalize(svhnData.getBatch()['x']).reshape((64,32*32*3))

    #dloss,gen_x,z_out_p = train_disc_gen_classifier(x_in,z_in)
    
    dloss= train_disc_gen_classifier_fast(x_in,z_in)


    if iteration % 10 == 0:
        print "iteration", iteration, "dloss", dloss, "gen_x mean", gen_x.mean()

    if iteration % 1000 == 0:
        print "dloss", dloss
        dloss,gen_x,z_out_p = train_disc_gen_classifier(x_in,z_in)
        plot_images(gen_x, "plots/" + slurm_name + "_gen.png")
        
        
        print "saving"
        save_network(gparams,dparams,"test")
        
        #plot_images(reconstruct(x_in).reshape((64,1,28,28)), "plots/" + slurm_name + "_rec.png")

        #NOT CORRECT INITIALLY
        #rec_loop = [x_in]
        #for b in range(0,9):
        #    rec_loop.append(reconstruct(rec_loop[-1]))
        #    rec_loop[-1][:,0:392] = x_in[:,0:392]
        #    plot_images(rec_loop[-1].reshape((64,1,28,28)), "plots/" + slurm_name + "_rec_" + str(b) +".png")

#         plot_images(x_in, "plots/" + slurm_name + "_original.png")

#         #p_chain = get_pchain(z_in)
#         new_z = rng.normal(size=(64,nl)).astype('float32')
#         for j in range(0,20):
#             new_x = func_z_to_x(new_z)
#             new_z = func_x_to_z(new_x)
#             print "printing element of p_chain", j
#             plot_images(new_x, "plots/" + slurm_name + "_pchain_" + str(j) + ".png")

#         new_z = rng.normal(size=(64,nl)).astype('float32')
#         for j in range(0,20):
#             new_x = func_z_to_x(new_z)
#             new_x = merge_images(new_x, x_in)
#             new_z = func_x_to_z(new_x)
#             plot_images(new_x, "plots/" + slurm_name + "_inpainting_" + str(j) + ".png")



iteration 0 dloss [array(0.4041844308376312, dtype=float32)] gen_x mean

NameError: name 'gen_x' is not defined

In [ ]:
save_network(gparams,dparams,"test-backup")

In [ ]:
#load_network("test")

In [ ]:
print "a"

In [ ]:
#svhnData.getBatch()['x'].shape

In [ ]:
#svhnData.train_X.shape

In [66]:
# super_samples = 2**13
# print "super_samples", super_samples

super_samples 8192


In [68]:
num_train = 1024
num_valid = 8192

In [69]:
soa_trainx = svhnData.train_X[:num_train]
soa_trainy = svhnData.train_Y[:num_train].reshape(num_train,)

soa_validx = svhnData.train_X[num_train:num_train+num_valid]
soa_validy = svhnData.train_Y[num_train:num_train+num_valid].reshape(num_valid,)

soa_testx = svhnData.train_X[num_train+num_valid:]
soa_testy = svhnData.train_Y[num_train+num_valid:].reshape(-1,)

In [70]:
print "soa_trainx", soa_trainx.shape
print "soa_validx", soa_validx.shape
print "soa_testx", soa_testx.shape

soa_trainx (1024, 3, 32, 32)
soa_validx (8192, 3, 32, 32)
soa_testx (564952, 3, 32, 32)


In [71]:
# x_batch = soa_trainx[:64]
# x_under_test = normalize(x_batch).reshape((64,32*32*3))
# z_under_test, zeo_under_test = func_x_to_z(x_under_test)
# print "z_under_test", z_under_test.shape
# print "zeo_under_test", zeo_under_test.shape

In [72]:
batch_size = 64
soa_trainz = np.ones((0,128))
soa_trainzeo = np.ones((0,8192))
for start in range(0, len(soa_trainx), batch_size):
    x_batch = soa_trainx[start:start + batch_size]
    x_under_test = normalize(x_batch).reshape((batch_size,32*32*3))
    z_under_test, zeo_under_test = func_x_to_z(x_under_test)
    soa_trainz = np.vstack([soa_trainz,z_under_test])
    soa_trainzeo = np.vstack([soa_trainzeo,zeo_under_test])

In [73]:
soa_trainz = soa_trainz.astype(np.float32)
soa_trainzeo = soa_trainzeo.astype(np.float32)
print "soa_trainz", soa_trainz.shape
print "soa_trainzeo", soa_trainzeo.shape

soa_trainz (1024, 128)
soa_trainzeo (1024, 8192)


In [81]:
soa_validz = np.ones((0,128))
soa_validzeo = np.ones((0,8192))
for start in range(0, len(soa_validx), batch_size):
    x_batch = soa_validx[start:start + batch_size]
    x_under_test = normalize(x_batch).reshape((batch_size,32*32*3))
    z_under_test, zeo_under_test = func_x_to_z(x_under_test)
    soa_validz = np.vstack([soa_validz,z_under_test])
    soa_validzeo = np.vstack([soa_validzeo,zeo_under_test])


In [82]:
soa_validz = soa_validz.astype(np.float32)
soa_validzeo = soa_validzeo.astype(np.float32)
print "soa_validz", soa_validz.shape
print "soa_validzeo", soa_validzeo.shape

soa_validz (8192, 128)
soa_validzeo (8192, 8192)


In [124]:
soa_testz = np.ones((0,128))
soa_testzeo = np.ones((0,8192))
for start in range(0, batch_size*1000, batch_size):
    x_batch = soa_testx[start:start + batch_size]
    x_under_test = normalize(x_batch).reshape((batch_size,32*32*3))
    z_under_test, zeo_under_test = func_x_to_z(x_under_test)
    soa_testz = np.vstack([soa_testz,z_under_test])
    soa_testzeo = np.vstack([soa_testzeo,zeo_under_test])

KeyboardInterrupt: 

In [125]:
soa_testz = soa_testz.astype(np.float32)
soa_testzeo = soa_testzeo.astype(np.float32)
print "soa_testz", soa_testz.shape
print "soa_testzeo", soa_testzeo.shape

soa_testz (42752, 128)
soa_testzeo (42688, 8192)


In [149]:
import collections
collections.Counter(soa_trainy)

Counter({0: 179,
         1: 137,
         2: 116,
         3: 105,
         4: 102,
         5: 80,
         6: 89,
         7: 58,
         8: 75,
         9: 83})

In [17]:
from joelearn.embeddings import tsne

In [ ]:
# coor_tsne = tsne.tsne(soa_trainzeo[:2048, :1024])
# plt.rcParams['figure.figsize'] = (15, 8)
# plt.scatter(coor_tsne[:, 0], coor_tsne[:, 1], c=soa_trainy[:2048], marker="o")

# for i, txt in enumerate(coor_tsne):
#     plt.annotate(soa_trainy[i], (txt[0],txt[1]))

In [251]:
from lasagne.layers import Conv2DLayer, InputLayer, ConcatLayer, DenseLayer, Pool2DLayer, FlattenLayer

input_var = T.matrix('inputs')

input_shape = (None, 8192)
net = net_in = InputLayer(shape=input_shape, input_var=input_var)
print net.output_shape

# net = lasagne.layers.batch_norm(DenseLayer(net, num_units=100))
# print net.output_shape

# net = lasagne.layers.batch_norm(DenseLayer(net, num_units=100))
# print net.output_shape

# net = lasagne.layers.batch_norm(DenseLayer(net, num_units=100))
# print net.output_shape

net = lasagne.layers.batch_norm(DenseLayer(net, num_units=100))
print net.output_shape

#net = ConcatLayer([net_1,net_2])

net = DenseLayer(net, num_units=10, nonlinearity=lasagne.nonlinearities.softmax)
print net.output_shape

(None, 8192)
(None, 100)
(None, 10)


In [252]:
output_shape = lasagne.layers.get_output_shape(net)
print "input_shape:",input_shape,"-> output_shape:",output_shape

input_shape: (None, 8192) -> output_shape: (None, 10)


In [253]:
output = lasagne.layers.get_output(net)
y = T.argmax(output, axis=1)
f_predict = theano.function([input_var], y)

In [254]:
#Training function
target_var = T.ivector('target')
target_one_hot = T.extra_ops.to_one_hot(target_var, 10)
loss = lasagne.objectives.categorical_crossentropy(output, target_one_hot)
loss = loss.mean()

#create lr variable so we can adjust the learning rate without compiling
lr = theano.shared(np.array(0.001, dtype=theano.config.floatX))

params = lasagne.layers.get_all_params(net, trainable=True)
updates = lasagne.updates.rmsprop(loss, params, learning_rate=lr)
#updates = lasagne.updates.nesterov_momentum(loss, params, learning_rate=lr, momentum=0.9)

f_train = theano.function([input_var, target_var], loss, updates=updates)
print "DONE building training functions"

DONE building training functions


In [255]:
import time
# train model
batch_size = 64
print "batch_size:",batch_size

stats = []
for i in range(0,2000):
    start_time = time.time()
    trainerror = []
    
    #Shuffle batches!
    order = range(0,len(soa_trainx))
    random.shuffle(order)
    z_train = soa_trainzeo[order]
    t_train = soa_trainy[order]
    
    #Start batch training!
    for start in range(0, len(soa_trainx), batch_size):
        
        z_batch = z_train[start:start + batch_size]
        t_batch = t_train[start:start + batch_size]
    
        #z_batch = np.random.normal(z_batch, scale=0.0001).astype(np.float32)

        
        cost = f_train(z_batch, t_batch)
        
        trainerror.append(cost)
        #print "batch error: %.5f" % cost
    elapsed_time = time.time() - start_time
    
    
    if i % 5 == 0:
        ## Calculate valid and test error 
        pred = np.array([])
        for start in range(0, len(soa_validx), batch_size):   #len(soa_testx)
            x_batch = soa_validzeo[start:start + batch_size]
            pred = np.append(pred,f_predict(x_batch))
        validacc = np.mean(pred == soa_validy)

        pred = np.array([])
        for start in range(0, len(soa_testx), batch_size):   #len(soa_testx)
            x_batch = soa_testzeo[start:start + batch_size]
            pred = np.append(pred,f_predict(x_batch))
        testacc = np.mean(pred == soa_testy[:len(pred)])

        epocherror = np.mean(trainerror)
        stats.append((validacc,epocherror, testacc))

        print "iteration: %d, trainerror: %.5f, validacc: %.5f, testacc: %.5f, seconds: %d" % (i, epocherror,validacc, testacc, elapsed_time)

batch_size: 64
iteration: 0, trainerror: 2.12699, validacc: 0.46753, testacc: 0.46835, seconds: 0
iteration: 5, trainerror: 0.25929, validacc: 0.60767, testacc: 0.61584, seconds: 0
iteration: 10, trainerror: 0.03115, validacc: 0.64160, testacc: 0.65105, seconds: 0
iteration: 15, trainerror: 0.00504, validacc: 0.64954, testacc: 0.66536, seconds: 0
iteration: 20, trainerror: 0.00230, validacc: 0.65234, testacc: 0.66815, seconds: 0
iteration: 25, trainerror: 0.00139, validacc: 0.65442, testacc: 0.66920, seconds: 0
iteration: 30, trainerror: 0.00115, validacc: 0.65381, testacc: 0.66906, seconds: 0
iteration: 35, trainerror: 0.00086, validacc: 0.65491, testacc: 0.66918, seconds: 0
iteration: 40, trainerror: 0.00061, validacc: 0.65625, testacc: 0.67073, seconds: 0
iteration: 45, trainerror: 0.00049, validacc: 0.65601, testacc: 0.67054, seconds: 0
iteration: 50, trainerror: 0.00049, validacc: 0.65674, testacc: 0.67103, seconds: 0
iteration: 55, trainerror: 0.00039, validacc: 0.65491, testacc:

KeyboardInterrupt: 

In [256]:
pred = np.array([])
for start in range(0, len(soa_trainzeo), batch_size):   #len(soa_testx)
    zeo_batch = soa_trainzeo[start:start + batch_size]
    pred = np.append(pred,f_predict(zeo_batch))
trainacc = np.mean(pred == soa_trainy)

In [257]:
trainacc

1.0

In [262]:
pred = np.array([])
for start in range(0, len(soa_testzeo), batch_size):   #len(soa_testx)
    zeo_batch = soa_testzeo[start:start + batch_size]
    pred = np.append(pred,f_predict(zeo_batch))
testacc = np.mean(pred == soa_testy[:len(soa_testzeo)])

In [263]:
testacc

0.671664167916042